### 0. 필요 파일 목록

* 'nst_{fish}_trend_2025-01-30.csv'
* 'forecast_agg.csv'
* 'ikh_item_price_2025-01-30.csv'

### 1. 경제지표 데이터 전처리

filled_economic_indicators.csv 생성

#### 1-1. yfinance에서 데이터 불러오기

* 입력 파일 :  없음
* 출력 파일 : '_economic_indicators_.csv'

In [312]:
import pandas as pd
from datetime import datetime
import yfinance as yf

# 시작일과 종료일 설정
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# 데이터를 저장할 빈 데이터프레임 생성
df_final = pd.DataFrame()

# 각 지표의 티커 심볼 정의
tickers = {
    'USD/KRW': 'KRW=X',  # 원달러 환율
    'KOSPI': '^KS11',  # KOSPI
    'WTI': 'CL=F',  # WTI 원유 선물
    'VIX': '^VIX',  # VIX 지수
    'Gold': 'GC=F',  # 금 선물
    'Silver': 'SI=F',  # 은 선물
    'MOVE' : '^MOVE' # MOVE Index
}

# 각 티커에 대해 데이터 다운로드
for name, ticker in tickers.items():
    try:
        df = yf.download(ticker, start=start_date, end=end_date, interval="1d", progress=False)
        # 종가만 사용
        df_final[name] = df['Close']
    except Exception as e:
        print(f"Error downloading {name}: {e}")

# 결측치 처리
df_final = df_final.fillna(method='ffill')

# 데이터 확인
print("\nFirst few rows of the data:")
print(df_final.head())

# 기본 통계 확인
print("\nBasic statistics:")
print(df_final.describe())

# CSV 파일로 저장
output_filename = '_economic_indicators_.csv'
df_final.to_csv(output_filename)
print(f"\nData saved to {output_filename}")


First few rows of the data:
                USD/KRW        KOSPI        WTI        VIX         Gold  \
Date                                                                      
2015-01-01  1092.699951          NaN        NaN        NaN          NaN   
2015-01-02  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
2015-01-05  1111.000000  1915.750000  50.040001  19.920000  1203.900024   
2015-01-06  1108.500000  1882.449951  47.930000  21.120001  1219.300049   
2015-01-07  1097.300049  1883.829956  48.650002  19.309999  1210.599976   

               Silver       MOVE  
Date                              
2015-01-01        NaN        NaN  
2015-01-02  15.734000  70.000000  
2015-01-05  16.179001  71.500000  
2015-01-06  16.603001  84.900002  
2015-01-07  16.510000  85.199997  

Basic statistics:
           USD/KRW        KOSPI          WTI          VIX         Gold  \
count  2626.000000  2625.000000  2625.000000  2625.000000  2625.000000   
mean   1198.244361  2373.886004  

C:\Users\user\AppData\Local\Temp\ipykernel_39148\280239614.py:33: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_final = df_final.fillna(method='ffill')


#### 1-2. 경제지표 데이터 결측치 채우기 

* 입력 파일 : '_economic_indicators_.csv'
* 출력 파일 : 'filled_economic_indicators_.csv'

In [313]:
import pandas as pd
import datetime

# 데이터 로드
df = pd.read_csv('_economic_indicators_.csv', encoding='utf-8')

# Date 컬럼을 datetime으로 변환
df['Date'] = pd.to_datetime(df['Date'])

# 시작일과 마지막일 추출
start_date = df['Date'].min()
end_date = df['Date'].max()

# 모든 날짜가 포함된 데이터프레임 생성
all_dates = pd.DataFrame(
    {'Date': pd.date_range(start_date, end_date, freq='D')}
)

# 기존 데이터와 병합
filled_df = pd.merge(all_dates, df, on='Date', how='left')

# 결측치를 이전 값으로 채우기
filled_df = filled_df.ffill()

# 결과 저장
filled_df.to_csv('filled_economic_indicators.csv', index=False)

# 결과 확인
print("처리된 데이터 샘플:")
print(filled_df.head())
print("\n결측치 확인:")
print(filled_df.isnull().sum())

처리된 데이터 샘플:
        Date      USD/KRW        KOSPI        WTI        VIX         Gold  \
0 2015-01-01  1092.699951          NaN        NaN        NaN          NaN   
1 2015-01-02  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
2 2015-01-03  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
3 2015-01-04  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
4 2015-01-05  1111.000000  1915.750000  50.040001  19.920000  1203.900024   

      Silver  MOVE  
0        NaN   NaN  
1  15.734000  70.0  
2  15.734000  70.0  
3  15.734000  70.0  
4  16.179001  71.5  

결측치 확인:
Date       0
USD/KRW    0
KOSPI      1
WTI        1
VIX        1
Gold       1
Silver     1
MOVE       1
dtype: int64


#### 1-3. 어종별 경제지표 칼럼 생성

* 입력 파일 : 'filled_economic_indicators_.csv'
* 출력 파일 : 'expanded_economic_indicators_.csv'

In [314]:
import pandas as pd

# 파일 경로 설정
file_path = "filled_economic_indicators.csv"  # 원본 데이터 파일
output_file = "expanded_economic_indicators.csv"  # 저장될 파일

# 데이터 로드
df = pd.read_csv(file_path)

# 복사할 경제 지표 리스트
economic_indicators = ["KOSPI", "USD/KRW", "WTI", "VIX", "Gold", "Silver", "MOVE"]

# 수산물 리스트
fish_types = ["광어", "농어", "대게", "방어", "연어", "우럭", "참돔"]

# 새로운 칼럼 생성 (각 경제 지표를 수산물별로 복사)
for fish in fish_types:
    for indicator in economic_indicators:
        df[f"{fish}_{indicator}"] = df[indicator]

# 처리된 데이터 저장
df.to_csv(output_file, index=False)

# 변경된 데이터 확인
print("\n✅ 변경된 데이터 (상위 5개 행):")
print(df.head())

print(f"\n📁 처리된 파일이 '{output_file}'로 저장되었습니다!")



✅ 변경된 데이터 (상위 5개 행):
         Date      USD/KRW        KOSPI        WTI        VIX         Gold  \
0  2015-01-01  1092.699951          NaN        NaN        NaN          NaN   
1  2015-01-02  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
2  2015-01-03  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
3  2015-01-04  1093.599976  1926.439941  52.689999  17.790001  1186.000000   
4  2015-01-05  1111.000000  1915.750000  50.040001  19.920000  1203.900024   

      Silver  MOVE     광어_KOSPI   광어_USD/KRW  ...      우럭_Gold  우럭_Silver  \
0        NaN   NaN          NaN  1092.699951  ...          NaN        NaN   
1  15.734000  70.0  1926.439941  1093.599976  ...  1186.000000  15.734000   
2  15.734000  70.0  1926.439941  1093.599976  ...  1186.000000  15.734000   
3  15.734000  70.0  1926.439941  1093.599976  ...  1186.000000  15.734000   
4  16.179001  71.5  1915.750000  1111.000000  ...  1203.900024  16.179001   

   우럭_MOVE     참돔_KOSPI   참돔_USD/KRW     참돔_WT

### 2. 트렌드 데이터 어종별로 그룹화



#### 2-1. 그룹화 함수 생성 및 저장

* 입력 파일 : 'nst_{fish}_trend_2025-##-##.csv'
* 출력 파일 : '그룹화_nst_{fish}_trend.csv'

In [342]:
import pandas as pd

def process_age_groups(file_path, fish_type):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 연령대 그룹 매핑 딕셔너리 생성
    age_mapping = {
        '19_24': '20대',
        '25_29': '20대',
        '30_34': '30대',
        '35_39': '30대',
        '40_44': '40대',
        '45_49': '40대',
        '50_54': '50대',
        '55_59': '50대',
        '60_80': '60대이상'
    }
    
    # 원하는 연령대만 필터링
    df = df[df['age'].isin(age_mapping.keys())]
    
    # 새로운 연령대 컬럼 생성
    df['age_group'] = df['age'].map(age_mapping)
    
    # 일자별, 새로운 연령대별로 score 합산
    result = df.groupby(['date', 'name', 'age_group'])['score'].sum().reset_index()
    
    # 피벗 테이블로 변환하여 보기 좋게 정리
    pivot_result = result.pivot(index=['date', 'name'], 
                              columns='age_group', 
                              values='score').reset_index()
    
    # 컬럼 순서 정리
    column_order = [ 'date', 'name', '20대', '30대', '40대', '50대', '60대이상']
    pivot_result = pivot_result[column_order]
    
    return pivot_result



In [343]:
#  수산물 종류별 파일 경로 및 이름 설정
fish_files = {
    '광어': 'nst_광어_trend_2025-01-30.csv',
    '농어': 'nst_농어_trend_2025-01-30.csv',
    '대게': 'nst_대게_trend_2025-01-30.csv',
    '방어': 'nst_방어_trend_2025-01-30.csv',
    '연어': 'nst_연어_trend_2025-01-30.csv',
    '우럭': 'nst_우럭_trend_2025-01-30.csv',
    '참돔': 'nst_참돔_trend_2025-01-30.csv'
}

#  개별 파일 저장 + 결과 리스트로 출력 준비
all_results = {}

#  모든 파일 처리
for fish, file_path in fish_files.items():
    try:
        processed_df = process_age_groups(file_path, fish)

        # 개별 파일 저장
        output_filename = f'그룹화_nst_{fish}_trend.csv'
        processed_df.to_csv(output_filename, index=False)

        # 결과 리스트에 추가 (첫 3개 행만 저장)
        all_results[fish] = processed_df.head(3)

        print(f"✅ '{output_filename}' 저장 완료!")

    except Exception as e:
        print(f"❌ Error processing {fish}: {e}")

#  최종 처리된 결과 출력 (각각 3개 행만 출력)
print("\n📊 최종 처리된 결과 미리보기 (각각 상위 3개 행)\n")
for fish, df_sample in all_results.items():
    print(f"{fish} 데이터 샘플:")
    print(df_sample)
    print("-" * 50)  # 구분선 추가

✅ '그룹화_nst_광어_trend.csv' 저장 완료!
✅ '그룹화_nst_농어_trend.csv' 저장 완료!
✅ '그룹화_nst_대게_trend.csv' 저장 완료!
✅ '그룹화_nst_방어_trend.csv' 저장 완료!
✅ '그룹화_nst_연어_trend.csv' 저장 완료!
✅ '그룹화_nst_우럭_trend.csv' 저장 완료!
✅ '그룹화_nst_참돔_trend.csv' 저장 완료!

📊 최종 처리된 결과 미리보기 (각각 상위 3개 행)

광어 데이터 샘플:
age_group        date name       20대       30대       40대       50대     60대이상
0          2016-01-01   광어  3.716023  3.554474  3.326419  3.809975  1.532954
1          2016-01-02   광어  3.827210  3.382407  3.133596  4.049620  1.812825
2          2016-01-03   광어  3.682096  3.701303  3.488837  3.279367  1.677356
--------------------------------------------------
농어 데이터 샘플:
age_group        date name       20대       30대       40대       50대     60대이상
0          2016-01-01   농어  3.260213  3.581407  3.558843  5.390086  2.163250
1          2016-01-02   농어  3.018128  3.318280  3.222967  3.078885  1.804692
2          2016-01-03   농어  2.932536  3.082965  2.882074  4.802194  1.134115
--------------------------------------------------
대게 데

#### 2-2. merged_trends.csv 저장

* 입력 파일 : '그룹화_nst_{fish}_trend.csv'
* 출력 파일 : 'merged_trends.csv'

In [344]:
import pandas as pd

def merge_fish_trends(file_paths, output_path):
    """
    여러 개의 수산물 트렌드 데이터를 병합하여 날짜별로 정리하는 함수.
    
    Args:
        file_paths (list): CSV 파일 경로 리스트.
        output_path (str): 병합된 데이터를 저장할 파일 경로.
    
    Returns:
        pd.DataFrame: 병합된 데이터프레임.
    """
    # 첫 번째 파일 로드 및 'name' 컬럼 제거
    merged_df = pd.read_csv(file_paths[0], encoding='utf-8')
    merged_df = merged_df.drop(columns=['name'], errors='ignore')  # name 컬럼 삭제 (없어도 오류 방지)
    merged_df['date'] = pd.to_datetime(merged_df['date'])  # 날짜 형식 변환
    
    # 파일명에서 수산물 이름 정확히 추출
    fish_name = file_paths[0].split('_')[2]  # 파일명 예: '그룹화_nst_광어_trend_2025-01-17.csv' → '광어'
    
    # 컬럼명 변경 (예: '20대' → '광어_20대')
    merged_df = merged_df.rename(columns={col: f"{fish_name}_{col}" for col in merged_df.columns if col not in ['date']})

    # 나머지 파일들 병합
    for file_path in file_paths[1:]:
        df = pd.read_csv(file_path, encoding='utf-8')
        df = df.drop(columns=['name'], errors='ignore')  # 'name' 컬럼 삭제
        df['date'] = pd.to_datetime(df['date'])

        # 파일명에서 수산물 이름 정확히 추출
        fish_name = file_path.split('_')[2]  # '그룹화_nst_농어_trend_2025-01-17.csv' → '농어'
        
        # 컬럼명 변경 (예: '20대' → '농어_20대')
        df = df.rename(columns={col: f"{fish_name}_{col}" for col in df.columns if col not in ['date']})

        # 병합 (중복 컬럼 제거)
        merged_df = pd.merge(merged_df, df, on='date', how='outer', suffixes=(None, None))

        # 중복 컬럼 자동 제거 (e.g. '광어_20대_x', '광어_20대_y')
        merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

    # 날짜순 정렬
    result = merged_df.sort_values('date')

    # CSV 저장
    result.to_csv(output_path, index=False, encoding='utf-8')
    
    print(f"\n✅ 병합된 데이터가 '{output_path}'에 저장되었습니다!")
    return result




In [345]:
# 사용 예시:
files = [
    '그룹화_nst_광어_trend.csv', 
    '그룹화_nst_농어_trend.csv', 
    '그룹화_nst_대게_trend.csv', 
    '그룹화_nst_방어_trend.csv', 
    '그룹화_nst_연어_trend.csv', 
    '그룹화_nst_우럭_trend.csv', 
    '그룹화_nst_참돔_trend.csv'
]

# 병합 실행
merged_result = merge_fish_trends(files, 'merged_trends.csv')

# 결과 확인
print("\n✅ 병합된 데이터 (상위 5개 행):")
print(merged_result.head())



✅ 병합된 데이터가 'merged_trends.csv'에 저장되었습니다!

✅ 병합된 데이터 (상위 5개 행):
        date    광어_20대    광어_30대    광어_40대    광어_50대  광어_60대이상    농어_20대  \
0 2016-01-01  3.716023  3.554474  3.326419  3.809975  1.532954  3.260213   
1 2016-01-02  3.827210  3.382407  3.133596  4.049620  1.812825  3.018128   
2 2016-01-03  3.682096  3.701303  3.488837  3.279367  1.677356  2.932536   
3 2016-01-04  3.449136  3.227122  2.811551  3.672470  1.940197  2.730419   
4 2016-01-05  3.401118  3.322712  3.168678  2.483900  1.000000  2.676719   

     농어_30대    농어_40대    농어_50대  ...    우럭_20대    우럭_30대    우럭_40대    우럭_50대  \
0  3.581407  3.558843  5.390086  ...  3.628428  3.449493  3.129530  4.836146   
1  3.318280  3.222967  3.078885  ...  3.745991  3.395083  2.909397  4.233117   
2  3.082965  2.882074  4.802194  ...  3.453351  3.871255  3.303071  3.276261   
3  3.361940  3.029135  3.059708  ...  3.333151  3.069274  3.539530  4.207426   
4  3.040365  3.129464  3.153394  ...  3.304805  3.384335  2.707205  2.638036   

### 3. 기상 데이터 전처리

* 입력 파일 : 'forecast_agg.csv'
* 출력 파일 : 'weatherdata_processed.csv'


In [346]:
# 함수 정의

def create_weather_columns(df, station_cols, output_path):
   result_df = df.copy()
   result_df['일시'] = pd.to_datetime(result_df['일시'])
   
   final_df = pd.DataFrame({'일시': result_df['일시'].unique()})
   
   for station, columns in station_cols.items():
       station_data = result_df[result_df['지점'] == station].copy()
       
       for orig_col, new_col in columns:
           station_col = station_data[['일시', orig_col]].copy()
           final_df = pd.merge(final_df, station_col.rename(columns={orig_col: new_col}), 
                             on='일시', how='left')
   
   # 칼럼을 가나다순으로 정렬 ('일시' 컬럼은 첫번째로 유지)
   sorted_cols = ['일시'] + sorted([col for col in final_df.columns if col != '일시'])
   result = final_df[sorted_cols].sort_values('일시')
   
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result

In [347]:
# CSV 파일 읽기
df = pd.read_csv('forecast_agg.csv', encoding='utf-8')

# 피쳐 선택
station_columns = {
	22105: [
		('기온', '광어_기온_22105'),
		('기온', '농어_기온_22105'),
		('기온', '대게_기온_22105'),
		('파주기', '대게_파주기_22105'),
		('파주기', '방어_파주기_22105'),
		('기온', '연어_기온_22105'),
		('습도', '연어_습도_22105')
	],
    
	22107: [
		('수온', '광어_수온_22107'),
		('수온', '농어_수온_22107'),
		('수온', '방어_수온_22107'),
		('수온', '연어_수온_22107'),
		('수온', '참돔_수온_22107')
	],

	22186: [
		('습도', '광어_습도_22186'),
		('습도', '농어_습도_22186'),
		('기온', '우럭_기온_22186'),
		('수온', '우럭_수온_22186')
		],
        
	22188: [
		('수온', '대게_수온_22188'),
		('습도', '대게_습도_22188')
		],        

	22189: [
		('파주기', '우럭_파주기_22189')
		],       

	22190: [
		('파주기', '광어_파주기_22190'),
        ('파주기', '농어_파주기_22190'),
        ('기온', '방어_기온_22190'),
        ('습도', '방어_습도_22190'),
        ('파주기', '연어_파주기_22190'),
        ('습도', '우럭_습도_22190'),
        ('기온', '참돔_기온_22190'),
        ('습도', '참돔_습도_22190'),
        ('파주기', '참돔_파주기_22190')
		]  

	}

result = create_weather_columns(df, station_columns, 'weatherdata_processed.csv')

### 4. 변수 하나로 합치기

* 입력 파일 : 'expanded_economic_indicators_.csv',  'merged_trends.csv', 'weatherdata_processed.csv'
* 출력 파일 : 'merged_all_data.csv'

In [348]:
def merge_all_data(output_path='merged_all_data.csv'):
   # 각 파일 로드 및 전처리
   
   economic = pd.read_csv('expanded_economic_indicators.csv', encoding='utf-8')
   economic['Date'] = pd.to_datetime(economic['Date'])
   economic = economic.rename(columns={'Date': '날짜'})
   
   trends = pd.read_csv('merged_trends.csv', encoding='utf-8')
   trends['date'] = pd.to_datetime(trends['date'])
   trends = trends.rename(columns={'date': '날짜'})
   
   weather = pd.read_csv('weatherdata_processed.csv', encoding='utf-8')
   weather['일시'] = pd.to_datetime(weather['일시'])
   weather = weather.rename(columns={'일시': '날짜'})
   
   # 모든 날짜 추출
   all_dates = pd.concat([
        economic['날짜'], 
        trends['날짜'], 
       weather['날짜']
   ]).unique()
   
   # 날짜 기준 데이터프레임 생성
   date_df = pd.DataFrame({'날짜': all_dates})
   date_df = date_df.sort_values('날짜')
   
   # 데이터 병합
   dfs = [
		date_df, 
		economic, 
		trends,
		weather
   ]
   
   result = dfs[0]
   for df in dfs[1:]:
       result = pd.merge(result, df, on='날짜', how='left')
   
   # 결과 저장
   result.to_csv(output_path, index=False, encoding='utf-8')
   return result


In [349]:

result = merge_all_data()

### 5. 결측치 채우기

* 입력 파일 : 'merged_all_data.csv'
* 출력 파일 : 'filled_merged_all_data.csv'

In [350]:
import pandas as pd

def fill_missing_values(file_path, output_path):
    """
    CSV 파일에서 결측치를 `ffill`을 사용하여 채우되, 이전 값이 없으면 그대로 둠.
    
    Args:
        file_path (str): 입력 CSV 파일 경로.
        output_path (str): 처리된 데이터를 저장할 경로.
    
    Returns:
        pd.DataFrame: 결측치가 채워진 데이터프레임.
    """
    
    # CSV 파일 로드
    df = pd.read_csv(file_path)

    # 결측치 확인 (처리 전)
    missing_before = df.isnull().sum()

    # 결측치 `ffill`로 채우기 (이전 값이 없으면 그대로 둠)
    df.fillna(method='ffill', inplace=True)

    # 결측치 확인 (처리 후)
    missing_after = df.isnull().sum()

    # 변경된 결측치 개수 출력
    missing_summary = pd.DataFrame({'Before': missing_before, 'After': missing_after})
    print("\n🔍 결측치 처리 전후 비교:")
    print(missing_summary)

    # 처리된 데이터 저장
    df.to_csv(output_path, index=False)
    print(f"\n✅ 처리된 데이터가 '{output_path}'에 저장되었습니다.")

    return df




In [351]:
file_path = 'merged_all_data.csv'
output_path = 'filled_merged_all_data.csv'

fill_missing_values(file_path, output_path)


C:\Users\user\AppData\Local\Temp\ipykernel_39148\1915651589.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)



🔍 결측치 처리 전후 비교:
              Before  After
날짜                 0      0
USD/KRW            0      0
KOSPI              1      1
WTI                1      1
VIX                1      1
...              ...    ...
우럭_파주기_22189     494    355
참돔_기온_22190      459    342
참돔_수온_22107      210      0
참돔_습도_22190      461    342
참돔_파주기_22190     421    342

[120 rows x 2 columns]

✅ 처리된 데이터가 'filled_merged_all_data.csv'에 저장되었습니다.


,날짜,USD/KRW,KOSPI,WTI,VIX,Gold,Silver,MOVE,광어_KOSPI,광어_USD/KRW,...,연어_습도_22105,연어_파주기_22190,우럭_기온_22186,우럭_수온_22186,우럭_습도_22190,우럭_파주기_22189,참돔_기온_22190,참돔_수온_22107,참돔_습도_22190,참돔_파주기_22190
0,2015-01-01,1092.699951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1092.699951,...,65.130435,NaN,NaN,NaN,NaN,NaN,NaN,16.382609,NaN,NaN
1,2015-01-02,1093.599976,1926.439941,52.689999,17.790001,1186.000000,15.734000,70.000000,1926.439941,1093.599976,...,60.083333,NaN,NaN,NaN,NaN,NaN,NaN,16.375000,NaN,NaN
2,2015-01-03,1093.599976,1926.439941,52.689999,17.790001,1186.000000,15.734000,70.000000,1926.439941,1093.599976,...,39.750000,NaN,NaN,NaN,NaN,NaN,NaN,16.283333,NaN,NaN
3,2015-01-04,1093.599976,1926.439941,52.689999,17.790001,1186.000000,15.734000,70.000000,1926.439941,1093.599976,...,54.000000,NaN,NaN,NaN,NaN,NaN,NaN,16.475000,NaN,NaN
4,2015-01-05,1111.000000,1915.750000,50.040001,19.920000,1203.900024,16.179001,71.500000,1915.750000,1111.000000,...,57.086957,NaN,NaN,NaN,NaN,NaN,NaN,16.475000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,2025-01-25,1433.930054,2536.800049,74.660004,14.850000,2777.300049,31.023001,86.750000,2536.800049,1433.930054,...,53.545455,6.004167,3.2375,9.683333,59.458333,5.3375,5.825,16.263636,59.458333,6.004167
3678,2025-01-26,1433.930054,2536.800049,74.660004,14.850000,2777.300049,31.023001,86.750000,2536.800049,1433.930054,...,53.545455,6.004167,3.2375,9.683333,59.458333,5.3375,5.825,16.263636,59.458333,6.004167
3679,2025-01-27,1427.630005,2536.800049,73.169998,17.900000,2737.500000,30.254000,95.050003,2536.800049,1427.630005,...,53.545455,6.004167,3.2375,9.683333,59.458333,5.3375,5.825,16.263636,59.458333,6.004167
3680,2025-01-28,1417.920044,2536.800049,73.769997,16.410000,2766.800049,30.726999,92.830002,2536.800049,1417.920044,...,53.545455,6.004167,3.2375,9.683333,59.458333,5.3375,5.825,16.263636,59.458333,6.004167


### 6. 타임래그 적용하기 

* 입력 파일 : 'filled_merged_all_data.csv'
* 출력 파일 : 'timelagged_features.csv'

#### 6-1. 설정된 타임래그 적용

In [352]:
# 데이터 로드
df = pd.read_csv('filled_merged_all_data.csv', encoding='utf-8')
df['date'] = pd.to_datetime(df['날짜'])

# 각 칼럼별 lag 일수 정의
lag_days = {

'광어_KOSPI' : 136,
'광어_USD/KRW' : 1,
'광어_WTI' : 1,
'광어_VIX' : 399,
'광어_Gold' : 314,
'광어_Silver' : 238,
'광어_MOVE' : 18,

'농어_KOSPI' : 179,
'농어_USD/KRW' : 1,
'농어_WTI' : 100,
'농어_VIX' : 391,
'농어_Gold' : 361,
'농어_Silver' : 290,
'농어_MOVE' : 1,

'대게_KOSPI' : 148,
'대게_USD/KRW' : 90,
'대게_WTI' : 91,
# '대게_VIX' : 
'대게_Gold' : 177,
'대게_Silver' : 177,
# '대게_MOVE' : 

'방어_KOSPI' : 282,
'방어_USD/KRW' : 387,
'방어_WTI' : 399,
'방어_VIX' : 133,
'방어_Gold' : 1,
'방어_Silver' : 1,
'방어_MOVE' : 381,

'연어_KOSPI' : 329,
'연어_USD/KRW' : 1,
'연어_WTI' : 199,
'연어_VIX' : 399,
'연어_Gold' : 1,
'연어_Silver' : 399,
'연어_MOVE' : 71,

'우럭_KOSPI' : 155,
'우럭_USD/KRW' : 121,
'우럭_WTI' : 14,
'우럭_VIX' : 38,
'우럭_Gold' : 146,
'우럭_Silver' : 125,
# '우럭_MOVE' : 

'참돔_KOSPI' : 399,
'참돔_USD/KRW' : 11,
'참돔_WTI' : 150,
# '참돔_VIX' : 
# '참돔_Gold' : 
# '참돔_Silver' : 
'참돔_MOVE' : 201,
 
'광어_20대' : 250,
'광어_30대' : 317,
'광어_40대' : 330,
'광어_50대' : 395,
'광어_60대이상' : 339,

'농어_20대' : 305,
'농어_30대' : 307,
'농어_40대' : 309,
'농어_50대' : 309,
'농어_60대이상' : 273,

'대게_20대' : 273,
'대게_30대' : 370,
'대게_40대' : 5,
'대게_50대' : 5,
'대게_60대이상' : 5,

'방어_20대' : 22,
'방어_30대' : 26,
'방어_40대' : 256,
'방어_50대' : 256,
'방어_60대이상' : 26,

'연어_20대' : 20,
'연어_30대' : 15,
'연어_40대' : 21,
'연어_50대' : 15,
'연어_60대이상' : 397,

'우럭_20대' : 22,
'우럭_30대' : 354,
'우럭_40대' : 354,
'우럭_50대' : 220,
'우럭_60대이상' : 219,

'참돔_20대' : 197,
'참돔_30대' : 167,
'참돔_40대' : 278,
'참돔_50대' : 193,
'참돔_60대이상' : 14,


'광어_기온_22105' : 97,
'광어_수온_22107' : 79,
'광어_습도_22186' : 349,
'광어_파주기_22190' : 103,

'농어_기온_22105' : 103,
'농어_수온_22107' : 81,
'농어_습도_22186' : 333,
'농어_파주기_22190' : 115,

'대게_기온_22105' : 150,
'대게_수온_22188' : 140,
'대게_습도_22188' : 355,
'대게_파주기_22105' : 174,

'방어_기온_22190' : 114,
'방어_수온_22107' : 118,
'방어_습도_22190' : 158,
'방어_파주기_22105' : 191,

'연어_기온_22105' : 394,
'연어_수온_22107' : 341,
'연어_습도_22105' : 8,
'연어_파주기_22190' : 9,

'우럭_기온_22186' : 118,
'우럭_수온_22186' : 113,
'우럭_습도_22190' : 159,
'우럭_파주기_22189' : 172,

'참돔_기온_22190' : 91,
'참돔_수온_22107' : 72,
'참돔_습도_22190' : 115,
'참돔_파주기_22190' : 107


}

# 각 칼럼에 대해 지정된 lag 적용
for col, lag in lag_days.items():
   if col in df.columns:
       df[f'{col}_{lag}'] = df[col].shift(lag)

# 결과 저장
df.to_csv('timelagged_features.csv', index=False, encoding='utf-8')

C:\Users\user\AppData\Local\Temp\ipykernel_39148\1547269827.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_{lag}'] = df[col].shift(lag)
C:\Users\user\AppData\Local\Temp\ipykernel_39148\1547269827.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_{lag}'] = df[col].shift(lag)
C:\Users\user\AppData\Local\Temp\ipykernel_39148\1547269827.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

#### 6-2. lag_1 파일 생성

In [353]:
# 데이터 로드
df = pd.read_csv('filled_merged_all_data.csv', encoding='utf-8')
df['date'] = pd.to_datetime(df['날짜'])

# 각 칼럼별 lag 일수 정의
lag_days = {

 
'광어_20대' : 1,
'광어_30대' : 1,
'광어_40대' : 1,
'광어_50대' : 1,
'광어_60대이상' : 1,

'농어_20대' : 1,
'농어_30대' : 1,
'농어_40대' : 1,
'농어_50대' : 1,
'농어_60대이상' : 1,

'대게_20대' : 1,
'대게_30대' : 1,
'대게_40대' : 1,
'대게_50대' : 1,
'대게_60대이상' : 1,

'방어_20대' : 1,
'방어_30대' : 1,
'방어_40대' : 1,
'방어_50대' : 1,
'방어_60대이상' : 1,

'연어_20대' : 1,
'연어_30대' : 1,
'연어_40대' : 1,
'연어_50대' : 1,
'연어_60대이상' : 1,

'우럭_20대' : 1,
'우럭_30대' : 1,
'우럭_40대' : 1,
'우럭_50대' : 1,
'우럭_60대이상' : 1,

'참돔_20대' : 1,
'참돔_30대' : 1,
'참돔_40대' : 1,
'참돔_50대' : 1,
'참돔_60대이상' : 1
}

# 각 칼럼에 대해 지정된 lag 적용
for col, lag in lag_days.items():
   if col in df.columns:
       df[f'{col}_{lag}'] = df[col].shift(lag)

# 결과 저장
df.to_csv('timelagged_features1.csv', index=False, encoding='utf-8')

#### 6-3. 트렌드 데이터 합치기

In [354]:
import pandas as pd

# 파일 경로 설정
file_path1 = "timelagged_features.csv"
file_path2 = "timelagged_features1.csv"
output_file = "final_timelagged_features.csv"  # 저장될 파일

# 데이터 로드
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)

# 공통 칼럼 확인
common_columns = set(df1.columns) & set(df2.columns)

# 중복되지 않는 칼럼만 선택
unique_columns_df1 = set(df1.columns) - common_columns
unique_columns_df2 = set(df2.columns) - common_columns

# 'date' 칼럼 포함하여 새로운 데이터프레임 생성
final_columns = ['date'] + list(unique_columns_df1) + list(unique_columns_df2)
df_final = pd.concat([df1[['date']], df1[list(unique_columns_df1)], df2[list(unique_columns_df2)]], axis=1)

# 'date' 칼럼을 제외한 나머지 칼럼을 가나다 순으로 정렬
sorted_columns = sorted([col for col in df_final.columns if col != 'date'])

# 'date' 칼럼을 앞에 두고 정렬된 칼럼 배치
df_final = df_final[['date'] + sorted_columns]

# 결과 저장
df_final.to_csv(output_file, index=False)

# 변경된 데이터 확인
print("\n✅ 변경된 데이터 (상위 5개 행):")
print(df_final.head())

print(f"\n📁 처리된 파일이 '{output_file}'로 저장되었습니다!")



✅ 변경된 데이터 (상위 5개 행):
         date  광어_20대_1  광어_20대_250  광어_30대_1  광어_30대_317  광어_40대_1  \
0  2015-01-01       NaN         NaN       NaN         NaN       NaN   
1  2015-01-02       NaN         NaN       NaN         NaN       NaN   
2  2015-01-03       NaN         NaN       NaN         NaN       NaN   
3  2015-01-04       NaN         NaN       NaN         NaN       NaN   
4  2015-01-05       NaN         NaN       NaN         NaN       NaN   

   광어_40대_330  광어_50대_1  광어_50대_395  광어_60대이상_1  ...  참돔_60대이상_1  참돔_60대이상_14  \
0         NaN       NaN         NaN         NaN  ...         NaN          NaN   
1         NaN       NaN         NaN         NaN  ...         NaN          NaN   
2         NaN       NaN         NaN         NaN  ...         NaN          NaN   
3         NaN       NaN         NaN         NaN  ...         NaN          NaN   
4         NaN       NaN         NaN         NaN  ...         NaN          NaN   

   참돔_KOSPI_399  참돔_MOVE_201  참돔_USD/KRW_11  참돔_WTI_150  참돔_기온_2

#### 6-4. 어종별 타임래그 데이터 분류

In [355]:
# 어종별 분류 

def split_by_fish():
    df = pd.read_csv('final_timelagged_features.csv', encoding='utf-8')
    fish_types = ['광어', '농어', '대게', '방어', '연어', '우럭', '참돔']
    
    for fish in fish_types:
        # Get date and fish-specific columns
        fish_cols = ['date'] + [col for col in df.columns if fish in col]
        fish_df = df[fish_cols]
        
        output_file = f'{fish}_timelagged_features.csv'
        fish_df.to_csv(output_file, index=False, encoding='utf-8')
        print(f'Created {output_file} with {len(fish_cols)} columns')

split_by_fish()

Created 광어_timelagged_features.csv with 22 columns
Created 농어_timelagged_features.csv with 22 columns
Created 대게_timelagged_features.csv with 20 columns
Created 방어_timelagged_features.csv with 22 columns
Created 연어_timelagged_features.csv with 22 columns
Created 우럭_timelagged_features.csv with 21 columns
Created 참돔_timelagged_features.csv with 19 columns


### 7. 가격 데이터 

* 입력 파일 : 'ikh_item_price_2025-01-30.csv
* 출력 파일 : 'item_price_lag.csv'

#### 7-1. 노량진 2층 버리기

* 입력 파일 : 'ikh_item_price_2025-01-30.csv
* 출력 파일 : 'item_price_deleted.csv'

In [356]:
import pandas as pd

# 파일 경로 설정
file_path = "ikh_item_price_2025-01-30.csv"  # 원본 데이터 파일
output_file = "item_price_deleted.csv"  # 저장될 파일

# 데이터 로드
df = pd.read_csv(file_path)

# 'market' 칼럼에서 '노량진 2층'인 행 제거
df = df[df['market'] != '노량진 2층']

# '노량진 1층'을 '노량진시장'으로 변경
df['market'] = df['market'].replace('노량진 1층', '노량진시장')

# 처리된 데이터 저장
df.to_csv(output_file, index=False)

# 변경된 데이터 확인
print("\n✅ 변경된 데이터 (상위 5개 행):")
print(df.head())

print(f"\n📁 처리된 파일이 '{output_file}'로 저장되었습니다!")



✅ 변경된 데이터 (상위 5개 행):
    priceDate  minPrice  avgPrice  maxPrice item market
0  2015-03-06     20000     20000     20000   농어  노량진시장
1  2015-03-14     25000     25000     25000   농어  노량진시장
2  2015-04-18     25000     27500     30000   농어  노량진시장
3  2015-04-28     25000     25000     25000   농어  노량진시장
4  2015-05-18     25000     25000     25000   농어  노량진시장

📁 처리된 파일이 'item_price_deleted.csv'로 저장되었습니다!


#### 7-2. 모든 날짜 생성 및 결측치 채우기

* 입력 파일 : 'item_price_deleted.csv
* 출력 파일 : 'item_price_filled_deleted.csv'

In [357]:
import pandas as pd

# 데이터 파일 로드
file_path = "item_price_deleted.csv"
df = pd.read_csv(file_path)

# 날짜 데이터 변환
df["priceDate"] = pd.to_datetime(df["priceDate"])

# item과 market의 조합별로 처리
filled_dfs = []

for (item, market), group in df.groupby(["item", "market"]):
    # 전체 날짜 범위 생성
    full_date_range = pd.date_range(start=group["priceDate"].min(), end=group["priceDate"].max())

    # 기존 데이터프레임을 날짜를 기준으로 리인덱싱
    group = group.set_index("priceDate").reindex(full_date_range).reset_index()
    group.rename(columns={"index": "priceDate"}, inplace=True)

    # item과 market 정보 채우기
    group["item"] = item
    group["market"] = market

    # 결측값을 앞의 값으로 채우기 (ffill)
    group.ffill(inplace=True)

    # 리스트에 추가
    filled_dfs.append(group)

# 모든 데이터를 합침
filled_df = pd.concat(filled_dfs, ignore_index=True)

# 결과 저장 (원하는 경로로 변경 가능)
output_path = "item_price_filled_deleted.csv"
filled_df.to_csv(output_path, index=False)

# 결과 확인
print(f"데이터가 {output_path} 파일로 저장되었습니다.")


데이터가 item_price_filled_deleted.csv 파일로 저장되었습니다.


#### 7-3. 1일 타임래그 생성

* 입력 파일 : 'item_price_filled_deleted.csv
* 출력 파일 : 'item_price_lag_filled_deleted.csv'

In [358]:
import pandas as pd

# 데이터 로드 (파일 경로는 적절하게 수정)
file_path = "item_price_filled_deleted.csv"
output_file = "item_price_lag_filled_deleted.csv"

df = pd.read_csv(file_path)

# 날짜 칼럼 이름을 'priceDate'에서 datetime 형식으로 변환
df['priceDate'] = pd.to_datetime(df['priceDate'])

# avgPrice에 1일 타임래그 적용
df['avgPrice_lag_1'] = df['avgPrice'].shift(1)

# 처리된 데이터 저장
df.to_csv(output_file, index=False)


### 8. 시장 데이터 원핫 인코딩

* 입력 파일 : 'item_price_lag_filled_deleted.csv'
* 출력 파일 : 'item_price_oneHot.csv'

In [359]:
import pandas as pd
import DateTime

def transform_market_data(file_path):
    # 데이터 읽기
    df = pd.read_csv(file_path)
    
    # 시장별 더미변수 생성 (0, 1로 인코딩)
    market_dummies = pd.get_dummies(df['market'], prefix='m').astype(int)
    
     # priceDate 칼럼명을 date로 변경
    df = df.rename(columns={'priceDate': 'date'})
    
    # 원본 데이터와 더미변수 결합
    result = pd.concat([
        df[['date', 'item']],  # 'priceDate' → 'date'로 변경됨
        market_dummies,
        df[['avgPrice', 'avgPrice_lag_1']]
    ], axis=1)
    
    # 날짜와 어종으로 정렬
    result = result.sort_values(['date', 'item'])
    
    # 변환된 데이터 저장
    output_file = 'item_price_oneHot.csv'
    result.to_csv(output_file, index=False)
    print(f"✅ 생성된 파일: {output_file}")
    print("\n🔍 처음 10개 컬럼:", result.columns[:10].tolist())
    
    return result




In [360]:
# Execute
df = transform_market_data('item_price_lag_filled_deleted.csv')

✅ 생성된 파일: item_price_oneHot.csv

🔍 처음 10개 컬럼: ['date', 'item', 'm_가락시장', 'm_강서농수산물시장', 'm_구리농수산물시장', 'm_노량진시장', 'm_마포농수산물시장', 'm_부산민락어민활어직판장', 'm_소래포구종합어시장', 'm_수원농수산물시장']


### 9. 가격데이터 어종별로 나누기

* 입력 파일 : 'item_price_oneHot.csv'
* 출력 파일 : '{fish}_price_oneHot.csv'

In [361]:
import pandas as pd

def split_market_data():
    df = pd.read_csv('item_price_oneHot.csv')
    
    for fish in df['item'].unique():
        fish_df = df[df['item'] == fish]
        output_file = f'{fish}_price_oneHot.csv'
      # fish_df = fish_df.drop('item', axis=1)
        fish_df.to_csv(output_file, index=False)
        print(f'{fish} 데이터 생성 완료: {len(fish_df)}개 행')



In [362]:
split_market_data()

대게 데이터 생성 완료: 24131개 행
광어 데이터 생성 완료: 29489개 행
농어 데이터 생성 완료: 28543개 행
연어 데이터 생성 완료: 27468개 행
참돔 데이터 생성 완료: 25041개 행
방어 데이터 생성 완료: 9715개 행
우럭 데이터 생성 완료: 5641개 행


### 10. 최종 데이터 합치기

* 입력 파일 : '{fish}_price_oneHot.csv', '{fish}_timelagged_features.csv'
* 출력 파일 : '{fish}_price_features.csv'

In [363]:
import pandas as pd

def merge_features_price(fish):
    # 두 데이터셋 로드
    features_df = pd.read_csv(f'{fish}_timelagged_features.csv')
    price_df = pd.read_csv(f'{fish}_price_oneHot.csv')
    
    # 날짜 컬럼명 통일
    price_df = price_df.rename(columns={'dmdpriceDate': 'date'})
    
    # 날짜 기준으로 데이터 병합
    merged_df = pd.merge(price_df, features_df, on='date', how='left')
    
    # CSV 파일로 저장
    output_file = f'{fish}_price_features.csv'
    merged_df.to_csv(output_file, index=False)
    
    print(f'생성된 파일: {output_file}')
    print(f'전체 컬럼 수: {len(merged_df.columns)}')


In [364]:
fish_list = ['광어', '농어', '대게', '방어', '연어', '우럭', '참돔']

for fish in fish_list:
    merge_features_price(fish)

생성된 파일: 광어_price_features.csv
전체 컬럼 수: 35
생성된 파일: 농어_price_features.csv
전체 컬럼 수: 35
생성된 파일: 대게_price_features.csv
전체 컬럼 수: 33
생성된 파일: 방어_price_features.csv
전체 컬럼 수: 35
생성된 파일: 연어_price_features.csv
전체 컬럼 수: 35
생성된 파일: 우럭_price_features.csv
전체 컬럼 수: 34
생성된 파일: 참돔_price_features.csv
전체 컬럼 수: 32


### 11. 최종 결측치 처리

#### 11-1. 결측치 없는 행만 남기기

In [365]:
import pandas as pd

def clean_fish_data(fish_list, input_path, output_path):
    """
    여러 수산물 데이터에서 NULL 값이 없는 첫 번째 행부터 마지막 행까지 필터링하여 저장하는 함수.

    Args:
        fish_list (list): 수산물 목록 (예: ['광어', '농어', ...])
        input_path (str): 원본 데이터 경로 패턴 (예: '{fish}_price_features.csv')
        output_path (str): 저장될 파일 경로 패턴 (예: 'cleaned_{fish}_price_features.csv')
    """
    for fish in fish_list:
        try:
            # 파일 경로 설정
            input_file = input_path.format(fish=fish)
            output_file = output_path.format(fish=fish)

            # 데이터 로드
            df = pd.read_csv(input_file)


            # 첫 번째로 NULL 값이 없는 행 찾기
            first_valid_index = df.dropna().index[1]

            # 마지막으로 NULL 값이 없는 행 찾기
            last_valid_index = df.dropna().index[-1]

            # NULL 값이 없는 첫 번째 행부터 마지막 행까지 데이터 선택
            df_cleaned = df.loc[first_valid_index:last_valid_index]

            # 결과 저장
            df_cleaned.to_csv(output_file, index=False)

            print(f"✅ 생성된 파일: {output_file} (전체 행 수: {len(df_cleaned)})")

        except Exception as e:
            print(f"❌ {fish} 데이터 처리 실패: {e}")




In [366]:
#  모든 수산물에 대해 반복 실행
fish_list = ['광어', '농어', '대게', '방어', '연어', '우럭', '참돔']
input_path = "{fish}_price_features.csv"
output_path = "notnull_{fish}_price_features.csv"  # {fish}_price_features_notnull.csv

clean_fish_data(fish_list, input_path, output_path)

✅ 생성된 파일: notnull_광어_price_features.csv (전체 행 수: 26555)
✅ 생성된 파일: notnull_농어_price_features.csv (전체 행 수: 25286)
✅ 생성된 파일: notnull_대게_price_features.csv (전체 행 수: 21516)
✅ 생성된 파일: notnull_방어_price_features.csv (전체 행 수: 9708)
✅ 생성된 파일: notnull_연어_price_features.csv (전체 행 수: 23975)
✅ 생성된 파일: notnull_우럭_price_features.csv (전체 행 수: 5630)
✅ 생성된 파일: notnull_참돔_price_features.csv (전체 행 수: 23354)


#### 11-2. 시장별 처음 판매일 삭제

In [367]:
import pandas as pd

def remove_missing_values(fish_list, file_path_pattern):
    """
    여러 수산물 데이터에서 결측치가 있는 행을 삭제하고 기존 파일에 덮어쓰는 함수.

    Args:
        fish_list (list): 수산물 목록 (예: ['광어', '농어', ...])
        file_path_pattern (str): 처리할 파일 경로 패턴 (예: 'notnull_{fish}_price_features.csv')
    """
    for fish in fish_list:
        try:
            # 파일 경로 설정
            file_path = file_path_pattern.format(fish=fish)

            # 데이터 로드
            df = pd.read_csv(file_path)

            # 결측치가 있는 행 제거
            df_cleaned = df.dropna()

            # 기존 파일에 덮어쓰기 저장
            df_cleaned.to_csv(file_path, index=False)

            print(f"✅ {fish}: 결측치 있는 행 제거 완료 (남은 행 수: {len(df_cleaned)})")

        except Exception as e:
            print(f"❌ {fish} 데이터 처리 실패: {e}")

# ✅ 모든 수산물에 대해 실행
fish_list = ['광어', '농어', '대게', '방어', '연어', '우럭', '참돔']
file_path_pattern = "notnull_{fish}_price_features.csv"

remove_missing_values(fish_list, file_path_pattern)


✅ 광어: 결측치 있는 행 제거 완료 (남은 행 수: 26555)
✅ 농어: 결측치 있는 행 제거 완료 (남은 행 수: 25286)
✅ 대게: 결측치 있는 행 제거 완료 (남은 행 수: 21516)
✅ 방어: 결측치 있는 행 제거 완료 (남은 행 수: 9708)
✅ 연어: 결측치 있는 행 제거 완료 (남은 행 수: 23975)
✅ 우럭: 결측치 있는 행 제거 완료 (남은 행 수: 5630)
✅ 참돔: 결측치 있는 행 제거 완료 (남은 행 수: 23354)


#### 11-1. 결측치 확인

In [368]:
import pandas as pd

def check_missing_values(fish_list, file_path_pattern):
    """
    여러 수산물 데이터에서 결측치 여부를 검사하는 함수.

    Args:
        fish_list (list): 수산물 목록 (예: ['광어', '농어', ...])
        file_path_pattern (str): 검사할 파일 경로 패턴 (예: 'notnull_{fish}_price_features.csv')
    """
    for fish in fish_list:
        try:
            # 파일 경로 설정
            file_path = file_path_pattern.format(fish=fish)

            # 데이터 로드
            df = pd.read_csv(file_path)

            # 결측치 확인
            missing_values = df.isnull().sum()
            total_missing = missing_values.sum()

            # 결과 출력
            if total_missing > 0:
                print(f"⚠️ {fish}: 총 {total_missing}개의 결측치 발견!")
                print(missing_values[missing_values > 0])
            else:
                print(f"✅ {fish}: 결측치 없음")

        except Exception as e:
            print(f"❌ {fish} 데이터 검사 실패: {e}")
            
# ✅ 모든 수산물에 대해 결측치 검사 실행
fish_list = ['광어', '농어', '대게', '방어', '연어', '우럭', '참돔']
file_path_pattern = "notnull_{fish}_price_features.csv"

check_missing_values(fish_list, file_path_pattern)


✅ 광어: 결측치 없음
✅ 농어: 결측치 없음
✅ 대게: 결측치 없음
✅ 방어: 결측치 없음
✅ 연어: 결측치 없음
✅ 우럭: 결측치 없음
✅ 참돔: 결측치 없음


### 12. 중간 과정 파일 삭제

In [369]:
import os

# 삭제할 어종 리스트
fish_list = ['광어', '농어', '대게', '방어', '연어', '우럭', '참돔']

remove_list = [
    '_economic_indicators_.csv',
	'filled_economic_indicators_.csv',
	'expanded_economic_indicators_.csv',
	f'그룹화_nst_{fish}_trend.csv',
	'merged_trends.csv',
	'weatherdata_processed.csv',
	'merged_all_data.csv',
	'filled_merged_all_data.csv',
	'timelagged_features.csv',
	'timelagged_features1.csv',
	'final_timelagged_features.csv',
	f'{fish}_timelagged_features.csv',
	'item_price_lag.csv',
	'item_price_lag_deleted.csv',
	'item_price_lag_filled_deleted.csv',
	'item_price_oneHot.csv',
	f'{fish}_price_oneHot.csv',
	f'{fish}_price_features.csv'
]

# 어종(fish)별 파일 추가
for fish in fish_list:
    remove_list.extend([
        f'그룹화_nst_{fish}_trend.csv',
        f'{fish}_timelagged_features.csv',
        f'{fish}_price_oneHot.csv',
        f'{fish}_price_features.csv',
    ])

# 리스트에 있는 파일들을 하나씩 삭제
for file in remove_list:
    if os.path.exists(file):  # 파일이 존재하는지 확인
        os.remove(file)
        print(f"Deleted: {file}")
    else:
        print(f"File not found: {file}")

Deleted: _economic_indicators_.csv
File not found: filled_economic_indicators_.csv
File not found: expanded_economic_indicators_.csv
Deleted: 그룹화_nst_참돔_trend.csv
Deleted: merged_trends.csv
Deleted: weatherdata_processed.csv
Deleted: merged_all_data.csv
Deleted: filled_merged_all_data.csv
Deleted: timelagged_features.csv
Deleted: timelagged_features1.csv
Deleted: final_timelagged_features.csv
Deleted: 참돔_timelagged_features.csv
File not found: item_price_lag.csv
File not found: item_price_lag_deleted.csv
Deleted: item_price_lag_filled_deleted.csv
Deleted: item_price_oneHot.csv
Deleted: 참돔_price_oneHot.csv
Deleted: 참돔_price_features.csv
Deleted: 그룹화_nst_광어_trend.csv
Deleted: 광어_timelagged_features.csv
Deleted: 광어_price_oneHot.csv
Deleted: 광어_price_features.csv
Deleted: 그룹화_nst_농어_trend.csv
Deleted: 농어_timelagged_features.csv
Deleted: 농어_price_oneHot.csv
Deleted: 농어_price_features.csv
Deleted: 그룹화_nst_대게_trend.csv
Deleted: 대게_timelagged_features.csv
Deleted: 대게_price_oneHot.csv
Deleted: 